In [1]:
import cv2
import numpy as np
import os
import json
from collections import defaultdict
from tqdm import tqdm_notebook

from IPython.display import clear_output

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 14) # (w, h)

In [2]:
def showInRow(list_of_images, titles = None, disable_ticks = False):
    count = len(list_of_images)
    for idx in range(count):
        subplot = plt.subplot(1, count, idx+1)
        if titles is not None:
            subplot.set_title(titles[idx])
          
        img = list_of_images[idx]
        cmap = 'gray' if (len(img.shape) == 2 or img.shape[2] == 1) else None
        subplot.imshow(img, cmap=cmap)
        if disable_ticks:
            plt.xticks([]), plt.yticks([])
    plt.show()

def draw_faces(img, faces, verbose=True):
    res_img = img.copy()
    
    for (x, y, w, h) in faces:
        cv2.rectangle(res_img, (x, y), (x+w, y+h), (0, 255, 0), 8)
    if verbose:
        plt.imshow(res_img)
        plt.show()
#     return res_img

def extract_faces(img, faces):
    res_imgs = []
    for (x, y, w, h) in faces:
        res_imgs.append(img[y:y+h, x:x+w])
    return res_imgs

def dlib_rect_to_bb(rect):
    """Take a bounding predicted by dlib and convert it
    to the format (x, y, w, h) as we would normally do
    with OpenCV
    
    Taken from https://github.com/jrosebr1/imutils/blob/master/imutils/face_utils/helpers.py
    """
    return rect_to_bb((rect.left(), rect.top(), rect.right(), rect.bottom()))

def rect_to_bb(rect):
    x1, y1, x2, y2 = rect
    return (x1, y1, x2-x1, y2-y1)

In [3]:
def read_json(path):
    with open(path) as input_file:
        return json.loads(input_file)

def write_json(obj, path, default=None):
    os.makedirs(os.path.split(path)[0], exist_ok=True)
    with open(path, 'w') as output_file:
        json.dump(obj, output_file, default=default, indent=4)

In [4]:
from utils import load_image, load_images, ImageLoader

## Test code

In [5]:
res_path = 'face_detection_res'

In [6]:
def convert_numpy_datatypes(o):
    if isinstance(o, np.int64) or isinstance(o, np.int32):
        return int(o)
    raise TypeError


def test_detector(detector, data, res_path='res.json', verbose=False):
    bboxes = {}
    for img_path, img in tqdm_notebook(data, desc='Detect faces'):
        faces = detector.detect(img)
        if isinstance(faces, tuple):
            faces = []
        elif isinstance(faces, np.ndarray):
            faces = faces.tolist()
        bboxes[img_path] = faces
        if verbose:
            draw_faces(img[:, :, ::-1], faces)
    write_json(bboxes, res_path, default=convert_numpy_datatypes)

In [7]:
data = ['data/test_img.jpg']
data = [(img_path, load_image(img_path)) for img_path in data]

In [8]:
data_path = os.path.join('data', 'FaceDetectionDataset')
data = ImageLoader(data_path, return_full_path=False)
len(data)

409

## Haar's feature selection

In [9]:
cascPath = 'data/haarcascade.xml'

In [10]:
class HaarDetector:
    """Haar feature selection."""
    def __init__(self, cascPath):
        self.model = cv2.CascadeClassifier(cascPath)
    
    def detect(self, img):
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return self.model.detectMultiScale(
            gray_img,
            scaleFactor=1.2,
            minNeighbors=7,
            minSize=(20, 20),
            flags = cv2.CASCADE_SCALE_IMAGE
        )
haar_detector = HaarDetector(cascPath)

In [11]:
test_detector(haar_detector, data, res_path=os.path.join(res_path, 'haar.json'))

## HOG

In [12]:
import dlib

class HOGDetector:
    """HOGDetector"""
    def __init__(self):
        self.model = dlib.get_frontal_face_detector()
    
    def detect(self, img):
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        face_rects = self.model(gray_img)
        return [dlib_rect_to_bb(face_rect) for face_rect in face_rects]

hog_detector = HOGDetector()

In [13]:
test_detector(hog_detector,
              data, 
              res_path=os.path.join(res_path, 'hog.json'),
              verbose=False)

## CNN (based on ResNet-34)
https://github.com/davisking/dlib-models

Doesn't work, caches results, which fills 8 gigabytes of RAM after processing 12 pictures.

In [9]:
import dlib

class ResNetDetector:
    "ResNetDetector"
    def __init__(self, path_to_weights):
        self.model = dlib.cnn_face_detection_model_v1(path_to_weights)
    
    def detect(self, img):
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        face_rects = self.model(gray_img, 1)
        face_rects = [rect.rect for rect in face_rects]
        return [dlib_rect_to_bb(face_rect) for face_rect in face_rects]

resnet_detector = ResNetDetector('data/model_weights/mmod_human_face_detector.dat')

In [19]:
dlib.DLIB_USE_CUDA

True

In [ ]:
test_detector(resnet_detector, data, res_path=os.path.join(res_path, 'resnet.json'))

## cvlib
https://www.cvlib.net/

In [14]:
import cvlib

class CVLibDetector:
    "CVLibDetector"
    def __init__(self):
        return
    
    def detect(self, img):
        face_rects, confidences = cvlib.detect_face(img)
        faces = [rect_to_bb(face_rect) for face_rect in face_rects]
        return faces

cvlib_detector = CVLibDetector()

Using TensorFlow backend.


In [15]:
test_detector(cvlib_detector, data, res_path=os.path.join(res_path, 'cvlib.json'), verbose=False)

## MTCNN

In [16]:
from mtcnn.mtcnn import MTCNN
model = MTCNN()
clear_output()

In [17]:
class MTCNNDetector:
    """MTCNNDetector"""
    def __init__(self):
        self.model = MTCNN()
    
    def detect(self, img):
        faces = self.model.detect_faces(img)
        faces = [x['box'] for x in faces]
        return faces

mtcnn_detector = MTCNNDetector()

In [18]:
test_detector(mtcnn_detector,
              data,
              res_path=os.path.join(res_path, 'mtcnn.json'),
              verbose=False)